# Descriptions
- Prediksi yang dilakukan adalah klasifikasi untuk target churn (1-Yes atau 0-No) dan Regresi untuk target tenure.
- Prediksi dengan klasifikasi menggunakan model machine learning
- Prediksi dengan regresi menggunakan model machine learning

# Import modules

In [1]:
import pandas as pd
import sys
import numpy
from sklearn import svm

numpy.set_printoptions(threshold=sys.maxsize)

# Initialize and load dataset
Terdapat tiga dataset, antara lain:
- EDA_df, yaitu dataset yang merupakan hasil dari eksplorasi terhadap fitur-fitur, yang telah bersih dari outlier, missing, duplikat, dan mismatch.
- feature_engineering_telco_customer_churn, yaitu dataset yang merupakan hasil dari feature engieering yang meliputi, feature transformation, feature extraction, dan feture selection.
- PCA_telco_customer_churn.csv adalah hasil reduki fitur menggunakan metode PCA (Principal Component Analysis).

## EDA

In [2]:
EDA_train_df = pd.read_csv('../datasets/train/EDA_train.csv')
EDA_test_df = pd.read_csv('../datasets/test/EDA_test.csv')

## Feature Engineering

In [3]:
feature_engineering_train_df = pd.read_csv('../datasets/train/feature_engineering_train.csv')
feature_engineering_test_df = pd.read_csv('../datasets/test/feature_engineering_test.csv')

## PCA

In [4]:
PCA_train_df = pd.read_csv('../datasets/train/PCA_train.csv')
PCA_test_df = pd.read_csv('../datasets/test/PCA_test.csv')

# Build model

## Support Vector Classifier

In [5]:
pd.set_option('display.max_columns', None)
EDA_train_df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,4223-BKEOR,0,0,0,0,21,0,0,0,1,0,1,0,0,1,1,0,3,64.85,1361.85,0
1,6035-RIIOM,0,0,0,0,54,0,1,1,0,1,0,0,1,1,2,1,0,97.20,5248.80,0
2,3797-VTIDR,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,23.45,23.45,1
3,2568-BRGYX,1,0,0,0,4,0,0,1,0,0,0,0,0,0,0,1,2,70.20,280.80,1
4,2775-SEFEE,1,0,0,0,0,0,1,0,1,1,0,1,0,0,2,1,0,61.90,0.00,0


### Select feature & target

#### Predicts target "churn"

In [11]:
X = EDA_train_df.loc[:, ['gender','senior_citizen','partner','dependents','tenure','phone_service', 'multiple_lines',
                         'internet_service','online_security','device_protection','tech_support','streaming_tv',
                         'streaming_movies','contract','paperless_billing','payment_method','monthly_charges','total_charges']
                     ]

y = EDA_train_df.loc[:, 'churn']

In [9]:
clf = svm.SVC()
clf.fit(X, y)
clf.predict(X)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Decision Tree

## Random Forest